In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

### The idea is to select the first 5 high-quality momentum stocks from the S&P500 that performed better the last 1 year using momentum strategy


In [ ]:
#import pandas_datareader as pdr
import datetime as dt
# Define the start and end dates for the data

end_date = dt.datetime(2023, 1, 1)
start_date = end_date - dt.timedelta(days=365)

# Use the DataReader function from pandas_datareader to download the data
#sp500_data = pdr.get_data_yahoo('^GSPC', start=start_date, end=end_date)

# Extract just the 'Close' column from the data
#sp500_close = sp500_data['Close']


In [ ]:
# Just download Apple data from Yahoo Finance and create a dataframe of the close price during the time frame.
Apple = yf.download('AAPL', start=start_date, end=end_date)
Apple = Apple[['Close']].copy()
Apple.dropna(inplace=True)
Apple

In [ ]:
Apple.info()

In [ ]:
# Retrieve all individual stocks from S&P500
import requests
import pandas as pd

# Define the URL to retrieve the list of stocks from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Use requests to retrieve the HTML data from the URL
html_data = requests.get(url).text

# Use pandas to read the HTML data and extract the table of stock data
sp500_table = pd.read_html(html_data, header=0)[0]

# Extract the 'Symbol' column from the table
sp500_symbols = sp500_table['Symbol'].tolist()
sp500_symbols

In [ ]:
stocks = yf.download(sp500_symbols, start=start_date, end=end_date)
stocks


In [134]:
stocks.index = pd.to_datetime(stocks.index)

In [ ]:
close = stocks[['Close']].copy()
close.info()

In [ ]:
close.iloc[-1].to_list()

In [ ]:
close = stocks.loc[:,'Close'].copy()
close.info()

In [ ]:
statistics = close.describe()
statistics.info()

In [ ]:
statistics

In [ ]:
Apple.mean()

In [ ]:
statistics.iloc[1,3]

In [ ]:
apple_close = close.AAPL.copy().to_frame()
apple_close

In [ ]:
apple_close['Open'] = apple_close.shift(periods=1)
apple_close.rename(columns={'AAPL':'Close'}, inplace=True)
apple_close["Diff"] = apple_close.Close.sub(apple_close.Open)
apple_close["% Daily Change"] = apple_close.Close.pct_change().mul(100)

apple_close.dropna(inplace=True)
apple_close

In [ ]:
# Apple stock, monthly returns
monthly_returns = apple_close.Close.resample('BM').last().pct_change().mul(100)
monthly_returns.dropna(inplace=True)
monthly_returns

In [ ]:
# Apple stock, yearly returns
yearly_returns = apple_close.Close.resample('Y').last().pct_change().mul(100)
yearly_returns.dropna(inplace=True)
yearly_returns

In [ ]:
yearly_returns.info()

#### Select only the first 5 highest-momentum stocks in the S&P 500. We will calculate the 1-year price return as the percentage change in the 'Close' column. Then we will sort the DataFrame by that new column.

In [ ]:
close.columns

In [ ]:
# Calculate the 1-year price return as the percentage change in the 'Close' price
yearly_returns = close.resample('Y').last().pct_change().mul(100)
yearly_returns

In [ ]:
yearly_returns.loc[:,'AAPL'][-1]

In [ ]:
# Resample or Summarize Time Series Data - Daily to Yearly
apple_data_yearly = close.AAPL.resample('Y').last().to_frame()
apple_data_yearly

In [ ]:
# Calculate the 1-year price return as the percentage change in the 'Close' price
last_year_return = (apple_data_yearly["AAPL"][-1] - apple_data_yearly["AAPL"][0]) / apple_data_yearly["AAPL"][0] * 100
last_year_return

In [ ]:
# Resample or Summarize Time Series Data - Daily to every 6 Months
apple_data_6m = close.AAPL.resample('6M').last()
apple_data_6m

In [ ]:
apple_data_6m.pct_change().mul(100)

In [ ]:
close.loc[:,'AAPL'][-1]

In [ ]:
tickers = close.columns.to_list()

### Lets select the first 5 stocks based on a very basic Momentum Strategy (only looking at the last year price return).
Remember we will calculate the one-year price return for each stock, and return the top 5 performers with the best price return in one year.

In [ ]:
data = {"Tickers": tickers, 
        "Close Price": close.iloc[-1].to_list(), 
        "One-Year Return": yearly_returns.iloc[-1].to_list(), 
        "Number of Shares To Buy": [100] * len(tickers)}

In [ ]:
initial_dataframe = pd.DataFrame(data, columns=["Tickers", "Close Price", "One-Year Return", "Number of Shares To Buy"])
initial_dataframe

In [ ]:
initial_dataframe.sort_values(by="One-Year Return", ascending=False, inplace=True, ignore_index=True)
initial_dataframe = initial_dataframe[:5]
initial_dataframe.reset_index(inplace=True,drop=True)
initial_dataframe

## Building a better (more realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "High quality" and "low quality" momentum stocks.

Lets start by building our DataFrame. You'll notice that i use the abbreviation hqm often, it stands for high-quality momentum.

In [ ]:
hqm_columns = ["Ticker", "Price", "Number of Shares to Buy", 
               "One-Year Price Return", "One-Year Return Percentile", 
               "Six-Month Price Return", "Six-Month Return Percentile",
               "Three-Month Price Return", "Three-Month Return Percentile",
               "One-Month Price Return", "One-Month Return Percentile"]


In [150]:

from scipy.stats import percentileofscore

def get_price_return_and_percentile(tickers_data: pd.DataFrame, ticker: str):
    """Receives only close price data"""   

    # Calculate the price return as the percentage change in the ticker column
    price_return = (tickers_data[ticker][-1] - tickers_data[ticker][0]) / tickers_data[ticker][0] * 100

    # Calculate the return percentile using the scipy.stats.percentileofscore function
    returns = tickers_data[ticker].pct_change().dropna() * 100
    return_percentile = percentileofscore(returns, price_return)

    # Round the return percentile to two decimal places
    return price_return, round(return_percentile, 2)


In [166]:
def download_close_price_data(tickers: list, start_date, end_date):
    """
    Download data for multiple tickers, and returns only Close price
    """
    # Use the Yahoo Finance library function to download the data for symbol/ticker
    stocks_data = yf.download(tickers, start=start_date, end=end_date, progress=False, repair = True)
    close = stocks_data['Close'].copy()
    return close


In [167]:
import datetime as dt
data = []

# Define start and end for 1-year period
end_date = dt.datetime(2023, 1, 1)
start_date = end_date - dt.timedelta(days=365)

one_year_data = download_close_price_data(sp500_symbols, start_date, end_date)

# Define start and end for 6-month period
end_date = dt.datetime(2023, 1, 1)
start_date = end_date - dt.timedelta(days=180)

six_month_data = download_close_price_data(sp500_symbols, start_date, end_date)

# Define start and end for 3-month period
end_date = dt.datetime(2023, 1, 1)
start_date = end_date - dt.timedelta(days=90)

three_month_data = download_close_price_data(sp500_symbols, start_date, end_date)

# Define start and end for 1-month period
end_date = dt.datetime(2023, 1, 1)
start_date = end_date - dt.timedelta(days=30)

one_month_data = download_close_price_data(sp500_symbols, start_date, end_date)


ERROR 
2 Failed downloads:
ERROR ['BF.B']: Exception('BF.B: No price data found, symbol may be delisted (1d 2022-01-01 00:00:00 -> 2023-01-01 00:00:00)')
ERROR ['BRK.B']: Exception('BRK.B: No timezone found, symbol may be delisted')
ERROR 
3 Failed downloads:
ERROR ['BF.B']: Exception('BF.B: No price data found, symbol may be delisted (1d 2022-07-05 00:00:00 -> 2023-01-01 00:00:00)')
ERROR ['BRK.B']: Exception('BRK.B: No timezone found, symbol may be delisted')
ERROR ['VRSN']: Exception('VRSN: No price data found, symbol may be delisted (1d 2022-07-05 00:00:00 -> 2023-01-01 00:00:00)')
ERROR 
2 Failed downloads:
ERROR ['BF.B']: Exception('BF.B: No price data found, symbol may be delisted (1d 2022-10-03 00:00:00 -> 2023-01-01 00:00:00)')
ERROR ['BRK.B']: Exception('BRK.B: No timezone found, symbol may be delisted')
ERROR 
2 Failed downloads:
ERROR ['BF.B']: Exception('BF.B: No price data found, symbol may be delisted (1d 2022-12-02 00:00:00 -> 2023-01-01 00:00:00)')
ERROR ['BRK.B']: Exc

In [168]:

for ticker in sp500_symbols:
    try:
        return_1year, percentile_1year = get_price_return_and_percentile(one_year_data, ticker)
        
        return_6month, percentile_6month = get_price_return_and_percentile(six_month_data, ticker)
        
        return_3month, percentile_3month = get_price_return_and_percentile(three_month_data, ticker)
        return_1month, percentile_1month = get_price_return_and_percentile(one_month_data, ticker)

        data.append({
            'Ticker': ticker, 
            'Price': close[ticker][-1],            
            'One-Year Price Return': return_1year, 
            'Six-Month Price Return': return_6month, 
            'Three-Month Price Return': return_3month, 
            'One-Month Price Return': return_1month,
            'One-Year Return Percentile': percentile_1year, 
            'Six-Month Return Percentile': percentile_6month, 
            'Three-Month Return Percentile': percentile_3month, 
            'One-Month Return Percentile': percentile_1month
            })
    except Exception as e:
        print(e)

In [169]:
stocks['Close'].copy().AAPL

Date
2021-12-31    177.570007
2022-01-03    182.009995
2022-01-04    179.699997
2022-01-05    174.919998
2022-01-06    172.000000
                 ...    
2022-12-23    131.860001
2022-12-27    130.029999
2022-12-28    126.040001
2022-12-29    129.610001
2022-12-30    129.929993
Name: AAPL, Length: 252, dtype: float64

In [170]:
hqm_dataframe = pd.DataFrame(data, columns=hqm_columns)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,MMM,119.919998,NaN,-32.530666,0.0,-7.189852,0.8,5.917680,100.00,-5.567367,0.00
1,AOS,57.240002,NaN,-31.506519,0.0,-0.261366,47.2,13.055502,100.00,-5.200396,0.00
2,ABT,109.790001,NaN,-21.037107,0.0,0.448305,67.2,10.319538,100.00,1.572768,78.95
3,ABBV,161.610001,NaN,19.339834,100.0,4.989286,100.0,16.837762,100.00,-1.252599,10.53
4,ACN,266.839996,NaN,-34.471157,0.0,-3.027226,6.4,0.736148,67.74,-11.091863,0.00
...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,128.080002,NaN,-6.189114,0.0,9.162192,100.0,17.386125,100.00,-1.233799,15.79
499,ZBRA,256.410004,NaN,-56.086660,0.0,-15.188696,0.8,-5.762796,1.61,-6.712502,0.00
500,ZBH,127.500000,NaN,1.699835,82.4,19.281505,100.0,18.803578,100.00,3.683824,100.00
501,ZION,49.160000,NaN,-23.474468,0.0,-3.758813,4.0,-5.841791,1.61,-5.913876,5.26
